In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.preprocessing import MinMaxScaler

In [70]:
# Load the CSV file
data = pd.read_csv('F:\\CS 325 Data Mining\\Datasets\\archive\\Heart.csv')
data.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


In [74]:
X= data.iloc[:,1:14]
Y= data.iloc[:,-1]
data['ChestPain'].replace(['typical', 'nontypical', 'nonanginal', 'asymptomatic'], [1,2,3,4], inplace=True)
data['Thal'].replace(['normal', 'fixed', 'reversable'], [1,2,3], inplace=True)
data['AHD'].replace(['No', 'Yes'], [0,1], inplace=True)
data.replace('NA', np.nan, inplace = True)
data.replace(' ', np.nan, inplace = True)
data.dropna( inplace = True, how = 'any')
data = data[data.Thal != '.']
data.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,2.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,1.0,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,3.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,1.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,1.0,0


In [75]:
# Separate the features (X) and the target variable (y)
X = data.iloc[:, :-1]  # Assuming the target variable is in the last column
y = data.iloc[:, -1]

In [76]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [81]:
# Scale the input features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [82]:
# Apply K-means clustering to determine the RBF centers
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(X_train)
centers = kmeans.cluster_centers_

In [83]:
# Calculate the RBF widths
distances = euclidean_distances(X_train, centers)
gamma = 1 / (2 * (distances.var()))

In [84]:
# Calculate the RBF activations for training and testing data
rbf_train = rbf_kernel(X_train, centers, gamma)
rbf_test = rbf_kernel(X_test, centers, gamma)

In [85]:
# Create an MLPRegressor model with RBF activation
rbf_model = make_pipeline(StandardScaler(), MLPRegressor(hidden_layer_sizes=(10,), activation='relu', solver='lbfgs'))

In [86]:
# Fit the model to the training data
rbf_model.fit(rbf_train, y_train)

C:\ProgramData\Anaconda3\ANACONDA3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('mlpregressor',
                 MLPRegressor(hidden_layer_sizes=(10,), solver='lbfgs'))])

In [87]:
# Make predictions on the test data
rbf_predictions = rbf_model.predict(rbf_test)

In [88]:
# Evaluate the model
mse = mean_squared_error(y_test, rbf_predictions)
print(f"Mean Squared Error (MSE): {mse}")

Mean Squared Error (MSE): 0.21120376088343587
